## Домашнее задание - 2

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('retail_train.csv')
data.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [3]:
data.shape

(2396804, 12)

In [4]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [5]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head()

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412..."


### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [6]:
def w_r_recommendation(items_weights, k=5):
    """Случайные рекоммендации"""
    items = items_weights['item_id'].values
    weights = items_weights['weights'].values
    recs = np.random.choice(items, size=k, replace=False, p=weights)
    
    return recs.tolist()

In [7]:
%%time

items_weights = data.groupby('item_id')['sales_value'].sum().reset_index()
items_weights['weights'] = items_weights['sales_value'] / items_weights['sales_value'].sum()
items_weights = items_weights.drop(['sales_value'], axis=1)

result['w_r_recommendation'] = result['user_id'].apply( \
    lambda x: w_r_recommendation(items_weights, k=5)
    )

CPU times: user 2.87 s, sys: 57.3 ms, total: 2.93 s
Wall time: 2.96 s


In [8]:
result.head()

,user_id,actual,w_r_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[969463, 1040915, 9802756, 5995312, 843656]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[6391532, 12263024, 933637, 1069506, 15972927]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[9926646, 1116686, 898847, 1098189, 831517]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[982799, 5570383, 6534178, 9553397, 918372]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[12384177, 1036559, 942817, 6534178, 864805]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [9]:
result = pd.read_csv('predictions_basic.csv')
result.head(1)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[8157117, 870291, 1846901, 891748, 1038560]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1082185, 981760, 1127831, 995242, 1098066]","[1082185, 981760, 1127831, 1098066, 961554]","[1082185, 981760, 1127831, 1098066, 961554]","[1082185, 1029743, 995785, 1004906, 1081177]"


In [10]:
result['actual'] = result['actual'].apply(lambda x: x.strip('[]').split())
result['random_recommendation'] = result['random_recommendation'].apply(
    lambda x: x.strip('[]').split(sep=', '))
result['popular_recommendation'] = result['popular_recommendation'].apply(
    lambda x: x.strip('[]').split(sep=', '))
result['itemitem'] = result['itemitem'].apply(lambda x: x.strip('[]').split(
    sep=', '))
result['cosine'] = result['cosine'].apply(lambda x: x.strip('[]').split(
    sep=', '))
result['tfidf'] = result['tfidf'].apply(lambda x: x.strip('[]').split(sep=', '))
result['own_purchases'] = result['own_purchases'].apply(
    lambda x: x.strip('[]').split(sep=', '))

In [11]:
result.head(1)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[8157117, 870291, 1846901, 891748, 1038560]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1082185, 981760, 1127831, 995242, 1098066]","[1082185, 981760, 1127831, 1098066, 961554]","[1082185, 981760, 1127831, 1098066, 961554]","[1082185, 1029743, 995785, 1004906, 1081177]"


In [12]:
def precision_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    flags = np.isin(bought_list, recommended_list)
    precision = flags.sum() / len(recommended_list)
        
    return precision

In [13]:
result['precision_random_recommendation'] = [precision_at_k(
    result['random_recommendation'][x], result['actual'][x], k=5) 
    for x in result.index]

res_ran_rec = (result['precision_random_recommendation'].sum() / 
    len(result['precision_random_recommendation']))
res_ran_rec

0.0005876591576885408

In [14]:
result['precision_popular_recommendation'] = [precision_at_k(
    result['popular_recommendation'][x], result['actual'][x], k=5) 
    for x in result.index]

res_pop_rec = (result['precision_popular_recommendation'].sum() / 
    len(result['precision_popular_recommendation']))
res_pop_rec

0.15523996082272282

In [15]:
result['precision_itemitem'] = [precision_at_k(
    result['itemitem'][x], result['actual'][x], k=5) 
    for x in result.index]

res_item = (result['precision_itemitem'].sum() / 
    len(result['precision_itemitem']))
res_item

0.15132223310479923

In [16]:
result['precision_cosine'] = [precision_at_k(
    result['cosine'][x], result['actual'][x], k=5) 
    for x in result.index]

res_cos = (result['precision_cosine'].sum() / 
    len(result['precision_cosine']))
res_cos

0.155435847208619

In [17]:
result['precision_tfidf'] = [precision_at_k(
    result['tfidf'][x], result['actual'][x], k=5) 
    for x in result.index]

res_tfidf = (result['precision_tfidf'].sum() / 
    len(result['precision_tfidf']))
res_tfidf

0.16121449559255635

In [18]:
result['precision_own_purchases'] = [precision_at_k(
    result['own_purchases'][x], result['actual'][x], k=5) 
    for x in result.index]

res_own_pur = (result['precision_own_purchases'].sum() / 
    len(result['precision_own_purchases']))
res_own_pur

0.20122428991185112

Лучшее качество показывает **own_purchases** алгоритм, с результатом:

**0.20122428991185112 **